In [ ]:
import time
import serial
import numpy as np
import pyqtgraph as pg
from PyQt5 import QtWidgets

# Initialize the Qt application
app = QtWidgets.QApplication([])

# Create a window and plot
win = pg.GraphicsLayoutWidget(show=True, title="Real-Time Plotting")
plots = [win.addPlot(title=f"{key}") for key in ["accelX", "accelY", "accelZ", "gyroX", "gyroY", "gyroZ"]]

# Initialize empty lists for sensor data
sensor_data = {
    "accelX": [],
    "accelY": [],
    "accelZ": [],
    "gyroX": [],
    "gyroY": [],
    "gyroZ": []
}

# Lines for plotting
lines = {key: plots[i].plot(pen='y') for i, key in enumerate(sensor_data.keys())}

# Maximum number of data points to store
MAX_DATA_POINTS = 50

# Establish Serial connection
ser = serial.Serial("/dev/cu.usbmodem11101", 115200)  # Adjust the port name and baud rate as needed
time.sleep(2)  # Time delay for Arduino Serial initialization

# Your data-generating loop
def update():
    try:
        # Read a line from the serial port
        arduinoData_string = ser.readline().decode('utf-8').strip()
        # print(f"Raw data: {arduinoData_string}")  # Debug print

        # Split the received data into separate values
        values = arduinoData_string.split(',')

        # Extract and store sensor data
        if len(values) == 6:
            sensor_data["accelX"].append(float(values[0].strip()))
            sensor_data["accelY"].append(float(values[1].strip()))
            sensor_data["accelZ"].append(float(values[2].strip()))
            sensor_data["gyroX"].append(float(values[3].strip()))
            sensor_data["gyroY"].append(float(values[4].strip()))
            sensor_data["gyroZ"].append(float(values[5].strip()))

            # Ensure the arrays do not exceed the maximum size
            for key in sensor_data.keys():
                if len(sensor_data[key]) > MAX_DATA_POINTS:
                    sensor_data[key].pop(0)

            # Update the plots
            for i, key in enumerate(sensor_data.keys()):
                lines[key].setData(sensor_data[key])
    except Exception as e:
        print(f"Error: {e}")
        pass

timer = pg.QtCore.QTimer()
timer.timeout.connect(update)
timer.start(100)  # Update every 100 ms

# Start the Qt event loop
QtWidgets.QApplication.instance().exec_()

ser.close()  # Close Serial connection when done